# Torch 사용 예
Transformer 기반의 pytorch 코드 중 모르는 부분 정리

In [2]:
import torch
import torch.nn as nn

## 1. nn.Embedding
임베딩 벡터를 사용하는 방법. 2가지
   1) embedding layer를 만들어서 훈련데이터로부터 처음부터 임베딩 벡터를 학습
   2) pre-trained word embedding을 가져와 사용하는 방법
nn.Embedding은 1번의 경우를 다룬다.

### 1.1 Embedding 구현

In [1]:
train_data = 'you need to know how to code'
word_set = set(train_data.split()) # 중복을 제거한 단어들의 집합인 단어 집합 생성.
vocab = {word: i+2 for i, word in enumerate(word_set)}  # 단어 집합의 각 단어에 고유한 정수 맵핑.
vocab['<unk>'] = 0
vocab['<pad>'] = 1
print(vocab)

{'how': 2, 'to': 3, 'need': 4, 'know': 5, 'code': 6, 'you': 7, '<unk>': 0, '<pad>': 1}


In [3]:
# 단어 집합의 크기만큼의 행을 가지는 테이블 생성.
embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],
                               [ 0.0,  0.0,  0.0],
                               [ 0.2,  0.9,  0.3],
                               [ 0.1,  0.5,  0.7],
                               [ 0.2,  0.1,  0.8],
                               [ 0.4,  0.1,  0.1],
                               [ 0.1,  0.8,  0.9],
                               [ 0.6,  0.1,  0.1]])

임의의 문장 you nead to run에 대해 앞서 만든 룩업 테이블을 통해 임베딩 벡터를 가져온다.

In [4]:
# 임의의 샘플 문장
sample = 'you need to run'.split()
idxes=[]

# 각 단어를 정수로 변환
for word in sample:
  try:
    idxes.append(vocab[word])
  except KeyError: # 단어 집합에 없는 단어일 경우 <unk>로 대체된다.
    idxes.append(vocab['<unk>'])
idxes = torch.LongTensor(idxes)

# 룩업 테이블
lookup_result = embedding_table[idxes, :] # 각 정수를 인덱스로 임베딩 테이블에서 값을 가져온다.
print(lookup_result)

tensor([[0.6000, 0.1000, 0.1000],
        [0.2000, 0.1000, 0.8000],
        [0.1000, 0.5000, 0.7000],
        [0.0000, 0.0000, 0.0000]])


### 1.2 nn.Embedding 사용
전처리는 동일한 과정을 거친다.

**문장과 사전 생성**

In [6]:
train_data = 'you need to know how to code'
word_set = set(train_data.split()) # 중복을 제거한 단어들의 집합인 단어 집합 생성.
vocab = {tkn: i+2 for i, tkn in enumerate(word_set)}  # 단어 집합의 각 단어에 고유한 정수 맵핑.
vocab['<unk>'] = 0
vocab['<pad>'] = 1

nn.Embedding을 사용해 **학습 가능**한 **임베딩 테이블** 생성 

In [9]:
embedding_layer = nn.Embedding(num_embeddings = len(vocab), 
                               embedding_dim = 3,
                               padding_idx = 1)

**nn.Embedding 인자**
- num_embeddings : 임베딩을 할 단어들의 개수. 다시 말해 단어 집합의 크기입니다.
- embedding_dim : 임베딩 할 벡터의 차원입니다. 사용자가 정해주는 하이퍼파라미터입니다.
- padding_idx : 선택적으로 사용하는 인자입니다. 패딩을 위한 토큰의 인덱스를 알려줍니다.

In [11]:
# 임베딩 테이블 생성
print(embedding_layer.weight)       

Parameter containing:
tensor([[ 0.6152,  0.2921, -0.6028],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.4120,  0.8522,  0.2752],
        [ 0.0034,  0.5262, -1.9511],
        [-0.1663, -0.7254,  1.1217],
        [-0.0802, -0.5327, -0.5864],
        [-1.6557, -0.1568, -0.3914],
        [-0.5595,  0.8364, -1.2726]], requires_grad=True)


In [13]:
embedding_layer(idxes)

tensor([[-0.5595,  0.8364, -1.2726],
        [-0.1663, -0.7254,  1.1217],
        [ 0.0034,  0.5262, -1.9511],
        [ 0.6152,  0.2921, -0.6028]], grad_fn=<EmbeddingBackward>)

# 2. torch.gather()
torch.gather(input, dim, index, out=None, sparse_grad=False) → Tensor
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2FnjDLW%2FbtqAs4zDa0Q%2FpBKwncy5GFhX23mTwJkalk%2Fimg.png)


In [18]:
t = torch.tensor([[1,2],[3,4]])
print(torch.gather(t, 0, torch.tensor([[0,0],[1,0]])))


tensor([[1, 2],
        [3, 2]])


## torch.multinomial
각 row가 num_samples 갯수 만큼 텐서 입력이 대응하는 row에 위치한 다형 확률 분포로부터 추출한다.
>torch.multinomial(input, num_samples, replacement=False, *, generator=None, out=None) → LongTensor


In [20]:
weights = torch.tensor([0, 10, 3, 0], dtype=torch.float) # create a tensor

In [21]:
torch.multinomial(weights, 2)

tensor([1, 2])

In [24]:
torch.multinomial(weights, 4)
# 3개 이상 뽑으려고 할때, 0이 2개이기 때문에 오류

RuntimeError: invalid multinomial distribution (with replacement=False, not enough non-negative category to sample)

In [26]:
torch.multinomial(weights, 4, replacement=True)

tensor([1, 1, 1, 1])

### torch.cat
주어진 차원 안에서 seq 텐서를 concat 한다. torch.split과 torch.chunk는 분리하는데 사용.
> torch.cat(tensors, dim=0, out=None) → Tensor
- **tensors** (sequence of Tensors) – any python sequence of tensors of the same type. Non-empty tensors provided must have the same shape, except in the cat dimension.

- **dim** (python:int, optional) – the dimension over which the tensors are concatenated

- **out** (Tensor, optional) – the output tensor.

In [27]:
x = torch.rand(2,3)

In [28]:
x

tensor([[0.3430, 0.8318, 0.3446],
        [0.5903, 0.7186, 0.2356]])

In [34]:
torch.cat((x,x,x),dim =0)

tensor([[0.3430, 0.8318, 0.3446],
        [0.5903, 0.7186, 0.2356],
        [0.3430, 0.8318, 0.3446],
        [0.5903, 0.7186, 0.2356],
        [0.3430, 0.8318, 0.3446],
        [0.5903, 0.7186, 0.2356]])

In [35]:
torch.cat((x, x, x), 1)

tensor([[0.3430, 0.8318, 0.3446, 0.3430, 0.8318, 0.3446, 0.3430, 0.8318, 0.3446],
        [0.5903, 0.7186, 0.2356, 0.5903, 0.7186, 0.2356, 0.5903, 0.7186, 0.2356]])